In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
from tqdm import tqdm
from transformers import T5Tokenizer, T5ForConditionalGeneration, AutoTokenizer, AutoModelForSeq2SeqLM

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def load_pretrained_model(model_name="google/pegasus-xsum"):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    model.to(device)
    model.eval()
    print(f"Loaded pretrained model: {model_name}")
    return tokenizer, model


In [ ]:
def generate_pretrained_summaries(texts, tokenizer, model, max_length=128, num_beams=4):
    summaries = []
    for text in tqdm(texts, desc="Generating Summaries"):
        inputs = tokenizer(
            text, return_tensors="pt", truncation=True, padding="max_length", max_length=512
        ).to(device)

        with torch.no_grad():
            outputs = model.generate(
                input_ids=inputs["input_ids"],
                attention_mask=inputs["attention_mask"],
                max_length=max_length,
                num_beams=num_beams,
                early_stopping=True,
                length_penalty=1.2,
            )
            summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
            summaries.append(summary)

    return summaries


In [ ]:
def compare_finetuned_and_pretrained(
    finetuned_model_path, pretrained_model_name, texts, tokenizer_finetuned, tokenizer_pretrained
):
    finetuned_model = T5ForConditionalGeneration.from_pretrained("t5-small")
    checkpoint = torch.load(finetuned_model_path, map_location=device)
    finetuned_model.load_state_dict(checkpoint["model_state_dict"])
    finetuned_model.to(device)
    finetuned_model.eval()

    print("Generating summaries with the fine-tuned model...")
    finetuned_summaries = []
    for text in tqdm(texts, desc="Fine-tuned Model Inference"):
        inputs = tokenizer_finetuned(
            f"summarize: {text}", return_tensors="pt", truncation=True, padding="max_length", max_length=512
        ).to(device)

        with torch.no_grad():
            outputs = finetuned_model.generate(
                input_ids=inputs["input_ids"],
                attention_mask=inputs["attention_mask"],
                max_length=128,
                num_beams=4,
                early_stopping=True,
            )
            summary = tokenizer_finetuned.decode(outputs[0], skip_special_tokens=True)
            finetuned_summaries.append(summary)

    tokenizer_pretrained, pretrained_model = load_pretrained_model(pretrained_model_name)

    print("Generating summaries with the pretrained model...")
    pretrained_summaries = generate_pretrained_summaries(texts, tokenizer_pretrained, pretrained_model)

    return finetuned_summaries, pretrained_summaries


In [ ]:
texts = [
    "Artificial intelligence is the simulation of human intelligence by machines. It enables tasks like speech recognition, decision-making, and translation.",
    "Machine learning is a subset of AI that uses algorithms to learn patterns in data and make predictions without explicit programming.",
]

finetuned_model_path = "/kaggle/input/summarizer-t5-small-epoch1and5/transformers/default/1/checkpoint_epoch_5.pt"
pretrained_model_name = "google/pegasus-xsum"

tokenizer_finetuned = T5Tokenizer.from_pretrained("t5-small")

finetuned_summaries, pretrained_summaries = compare_finetuned_and_pretrained(
    finetuned_model_path, pretrained_model_name, texts, tokenizer_finetuned, None
)

for i, text in enumerate(texts):
    print(f"\nInput {i + 1}: {text}")
    print(f"Fine-tuned Summary: {finetuned_summaries[i]}")
    print(f"Pretrained Summary: {pretrained_summaries[i]}")


/tmp/ipykernel_30/3861490637.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(finetuned_model_path, map_location=device)


Generating summaries with the fine-tuned model...


Fine-tuned Model Inference: 100%|██████████| 2/2 [00:01<00:00,  1.17it/s]


tokenizer_config.json:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

...

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/259 [00:00<?, ?B/s]

Loaded pretrained model: google/pegasus-xsum
Generating summaries with the pretrained model...


Generating Summaries: 100%|██████████| 2/2 [00:21<00:00, 10.61s/it]



Input 1: Artificial intelligence is the simulation of human intelligence by machines. It enables tasks like speech recognition, decision-making, and translation.
Fine-tuned Summary: Learn about Artificial Intelligence.
Pretrained Summary: Artificial intelligence (AI) is a branch of computer science that deals with the study of human intelligence.

Input 2: Machine learning is a subset of AI that uses algorithms to learn patterns in data and make predictions without explicit programming.
Fine-tuned Summary: Understand machine learning.
Pretrained Summary: Researchers at the Massachusetts Institute of Technology (MIT) have developed a machine learning algorithm that can predict whether a patient will be admitted to the hospital on time.
